# Wrangle and Analyze Data

In [12]:
import pandas as pd
import numpy as np
import requests
import tweepy as tweepy
import json
from timeit import default_timer as timer


## Gather Data

### Load the tweets from the archive

In [2]:
# Read in the twitter archive enhanced csv given to us
df_twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')
'{} tweets loaded from the archive'.format(len(df_twitter_archive))

'2356 tweets loaded from the archive'

### Get the image predictions

In [3]:
# Get the file from Udacity servers
image_predictions_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
image_predictions_response = requests.get(image_predictions_url);

# response status should be 200
print('Response Status {}'.format(image_predictions_response))

# save to file system
with open(image_predictions_url.split('/')[-1], mode='wb') as file:
    file.write(image_predictions_response.content)

# Load the dataframe
df_image_predictions = pd.read_csv('image-predictions.tsv', sep='\t')
'load {} image prediction from the tsv file'.format(len(df_image_predictions.head()))

Response Status <Response [200]>


'load 5 image prediction from the tsv file'

### Query tweets'status from Twitter

In [4]:
consumer_token = '1CWL9OEQCIo7TAmrrHtzTCDML'
consumer_secret = 'AeNMfj3cdGt7tAStopY5fUJRzLlFKml0zHA2P1n6OQ40IF3q3U'
access_token = '150976386-3w2ZXZE6gJaq4WjIH9wb9rOLLJ2WOswdiT0Kg1AH'
access_secret = '0kV0KVAV2cZxpei37mPxuV8QyRdumYnM13C0ZocFsOL7z'

auth = tweepy.OAuthHandler(consumer_token, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

In [10]:
df_list = []
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in df_twitter_archive.tweet_id.values:
        try:
            tweet = api.get_status(tweet_id,
                                  wait_on_rate_limit = True, wait_on_rate_limit_notify = True)
            #print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
#            favorite_count = tweet['favorite_count'] # How many favorites the tweet had
#            retweet_count = tweet['retweet_count'] # Count of the retweet 
#            df_list.append({'tweet_id': int(tweet_id),
#                        'favorite_count': int(favorite_count),
#                        'retweet_count': int(retweet_count)})

        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
#print('Retrieve {} status info'.format(len(fails_dict)))
print('Fail retrieval {}'.format(len(fails_dict)))

# Save the result in a file to avoid further API call
#df = pd.DataFrame(df_list, columns = ['tweet_id', 'favorite_count', 'retweet_count'])
#df.to_csv('tweet_json.txt', index=False)

Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Fail
Rate limit reached. Sleeping for: 535
Fail
Fail
Rate limit reached. Sleeping for: 664
1831.1260338929997
Fail retrieval 19


In [17]:
#len(fails_dict)

tweet_list = []

# Read the .txt file line by line into a list of dictionaries
for line in open('tweet_json.txt', 'r'):
    d = json.loads(line)
    #print(d)
    tweet_list.append({'tweet_id': d['id_str'],
                     'retweet_count': d['retweet_count'],
                     'favorite_count': d['favorite_count']})
    
# Save the result to a file to avoie further API call
df = pd.DataFrame(tweet_list, columns = ['tweet_id', 'favorite_count', 'retweet_count'])
df.to_csv('tweet_json.csv', index=False)  
len(df)

2337

In [18]:
df_tweet_data = pd.read_csv('tweet_json.csv')
'Load {} tweet status from Twitter'.format(len(df_tweet_data))

'Load 2337 tweet status from Twitter'

## Assess Data

In [151]:
df_twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')
df_image_predictions = pd.read_csv('image-predictions.tsv', sep='\t')
df_tweet_data = pd.read_csv('tweet_json.csv')

### Visual assessment of the 3 dataframes

In [152]:
df_twitter_archive.sample(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
114,870656317836468226,NaN,NaN,2017-06-02 15:00:16 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cody. He zoomed too aggressively and t...,NaN,NaN,NaN,https://www.gofundme.com/help-fix-codys-torn-a...,13,10,Cody,None,None,None,None
1372,702217446468493312,NaN,NaN,2016-02-23 19:44:20 +0000,"<a href=""http://twitter.com/download/iphone"" r...","I know it's tempting, but please stop sending ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/702217446...,9,10,None,None,None,None,None
801,772581559778025472,NaN,NaN,2016-09-04 23:46:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Guys this is getting so out of hand. We only r...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/772581559...,10,10,a,None,None,None,None
2051,671488513339211776,NaN,NaN,2015-12-01 00:38:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Julius. He's a cool dog. Carries seash...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/671488513...,8,10,Julius,None,None,None,None
856,764259802650378240,NaN,NaN,2016-08-13 00:38:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Kota and her son Benedict. She doesn't...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/764259802...,10,10,Kota,None,None,None,None
1130,729113531270991872,NaN,NaN,2016-05-08 00:59:46 +0000,"<a href=""http://twitter.com/download/iphone"" r...","""Challenge accepted""\n10/10 https://t.co/vNjvr...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/729113531...,10,10,None,None,None,None,None
2150,669683899023405056,NaN,NaN,2015-11-26 01:07:38 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Kloey. Her mother was a unicorn. 10/10...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/669683899...,10,10,Kloey,None,None,None,None
1648,683773439333797890,NaN,NaN,2016-01-03 22:14:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Buddy. He's gaining strength. Currentl...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/683773439...,9,10,Buddy,None,None,None,None
1171,720415127506415616,NaN,NaN,2016-04-14 00:55:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Garden's coming in nice this year. 10/10 https...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/720415127...,10,10,None,None,None,None,None
764,777953400541634568,NaN,NaN,2016-09-19 19:31:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Gerald. He's a fairly exot...,7.681934e+17,4.196984e+09,2016-08-23 21:09:14 +0000,https://twitter.com/dog_rates/status/768193404...,8,10,Gerald,doggo,None,None,None


In [153]:
df_image_predictions.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
901,700002074055016451,https://pbs.twimg.com/media/CbboKP4WIAAw8xq.jpg,1,Chihuahua,0.369488,True,schipperke,0.243367,True,pug,0.161614,True
265,670803562457407488,https://pbs.twimg.com/media/CU8sSAvXIAAB1Py.jpg,1,basenji,0.344101,True,Ibizan_hound,0.210282,True,toy_terrier,0.196279,True
27,666396247373291520,https://pbs.twimg.com/media/CT-D2ZHWIAA3gK1.jpg,1,Chihuahua,0.978108,True,toy_terrier,0.009397,True,papillon,0.004577,True
616,680191257256136705,https://pbs.twimg.com/media/CXCGVXyWsAAAVHE.jpg,1,Brittany_spaniel,0.733253,True,Welsh_springer_spaniel,0.251634,True,English_springer,0.009243,True
1719,819227688460238848,https://pbs.twimg.com/media/C157Oq3WQAEOyHm.jpg,1,Border_terrier,0.482452,True,German_shepherd,0.181082,True,Norwegian_elkhound,0.065257,True
1235,746507379341139972,https://pbs.twimg.com/media/Clwgf4bWgAAB15c.jpg,1,toy_poodle,0.508292,True,Lakeland_terrier,0.234458,True,affenpinscher,0.084563,True
1940,860924035999428608,https://pbs.twimg.com/media/C_KVJjDXsAEUCWn.jpg,2,envelope,0.933016,False,oscilloscope,0.012591,False,paper_towel,0.011178,False
65,667174963120574464,https://pbs.twimg.com/media/CUJIFoJWsAAL3Dc.jpg,1,toy_poodle,0.266437,True,Chihuahua,0.243223,True,bluetick,0.072806,True
2031,883117836046086144,https://pbs.twimg.com/media/DEF2-_hXoAAs62q.jpg,2,golden_retriever,0.949562,True,Labrador_retriever,0.045948,True,kuvasz,0.002471,True
1686,814638523311648768,https://pbs.twimg.com/media/C04taUjWIAA6Mo4.jpg,2,golden_retriever,0.650814,True,kuvasz,0.053281,True,cocker_spaniel,0.035440,True


In [154]:
df_tweet_data.sample(10)

,tweet_id,favorite_count,retweet_count
2226,667885044254572545,813,490
469,815390420867969024,10933,4113
1049,740676976021798912,18821,7158
153,860924035999428608,0,819
722,780858289093574656,7594,2159
1285,707420581654872064,2358,724
1501,690735892932222976,3933,1340
201,852226086759018497,20337,7006
451,817415592588222464,5912,1039
1336,703631701117943808,2693,721


**Tweeter Archive**:
* The format of values for columns `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp` and `timestamp` indicate that there's a problem of type (which would be clarified via programmatic assessemnt)
* Column `name` has some values to "None" which probalby means that the dog's name is not known => we should get null instead of "None" for those
* In columns `doggo`, `floofer`, `pupper` and `puppo` 'None' is displayed for null and when not null the value is the same as the column name. This looks like a tidiness issue: Each variable forms a column. Here the dog stage variable is spread over four columns.
* Column `source` contains markup making the reading less friendly.
* The presence of the colum `retweeted_status_id`, `retweeted_status_user_id` and `retweeted_status_timestamp` and the fact that they sometimes contain not null value indicate that there are record related to retweeted tweets which are not in the population we want analyze.

**General tidiness issue**:
* The three datasets should be combined into one since they are all part of the same observation unit.

### Programatic assessment on twitter archive

In [155]:
df_twitter_archive.info()
print("timestamp type is " + str(type(df_twitter_archive.dtypes.timestamp.kind)))
print("retweeted_status_timestamp type is " + str(type(df_twitter_archive.dtypes.timestamp.kind)))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [156]:
'There are {} duplicate in the twitter archive'.format(len(df_twitter_archive[df_twitter_archive.tweet_id.duplicated()]))

'There are 0 duplicate in the twitter archive'

In [157]:
print(df_twitter_archive.source.value_counts())
print("There's just four different sources")

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64
There's just four different sources


In [158]:
# Describe only numerator and denominator since the other numerical values are identifiers.
df_twitter_archive.describe()['rating_denominator'], df_twitter_archive.describe()['rating_numerator']

(count    2356.000000
 mean       10.455433
 std         6.745237
 min         0.000000
 25%        10.000000
 50%        10.000000
 75%        10.000000
 max       170.000000
 Name: rating_denominator, dtype: float64, count    2356.000000
 mean       13.126486
 std        45.876648
 min         0.000000
 25%        10.000000
 50%        11.000000
 75%        12.000000
 max      1776.000000
 Name: rating_numerator, dtype: float64)

In [159]:
df_twitter_archive.rating_denominator.value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [160]:
df_twitter_archive.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [161]:
# View tweets with decimals in rating in 'text' column
df_decimals = df_twitter_archive[df_twitter_archive.text.str.contains(r"(\d+\.\d*\/\d+)")]

ratings = df_decimals.text.str.extract('((?:\d+\.)?\d+)\/(\d+)', expand=True).rename(columns={0: 'extract_rating_numerator', 1: 'extract_rating_denominator'})
pd.concat([df_decimals[['tweet_id','rating_numerator']],ratings], axis=1)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


,tweet_id,rating_numerator,extract_rating_numerator,extract_rating_denominator
45,883482846933004288,5,13.5,10
340,832215909146226688,75,9.75,10
695,786709082849828864,75,9.75,10
763,778027034220126208,27,11.27,10
1689,681340665377193984,5,9.5,10
1712,680494726643068929,26,11.26,10


**Type issues**:
* Following columns should be `int64` or `string` instead `float64`: 
    - in_reply_to_status_id
    - in_reply_to_user_id
    - retweeted_status_id
    - retweeted_status_user_id
* Following colums should be of type `datetime` instead of `object`:
    - timestamp
    - retweeted_status_timestamp
* Column `source` should be category
    
**Value issues**:
* `rating_denominator` should always be 10.
* `rating_numerator` may be extracted from the text when decimals are present.
    

### Programatic assessment image prediction

In [162]:
df_image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [163]:
print('There are {} duplicates in picture predictions.'.format(len(df_image_predictions[df_image_predictions['tweet_id'].duplicated()])))
print('There are {} missing picture predictions.'.format(len(df_twitter_archive) - len(df_image_predictions)))

There are 0 duplicates in picture predictions.
There are 281 missing picture predictions.


In [164]:
df_image_predictions['jpg_url'].value_counts()

https://pbs.twimg.com/media/C2kzTGxWEAEOpPL.jpg                                            2
https://pbs.twimg.com/media/Ck2d7tJWUAEPTL3.jpg                                            2
https://pbs.twimg.com/media/CkNjahBXAAQ2kWo.jpg                                            2
https://pbs.twimg.com/media/CsGnz64WYAEIDHJ.jpg                                            2
https://pbs.twimg.com/media/C12x-JTVIAAzdfl.jpg                                            2
https://pbs.twimg.com/media/CzG425nWgAAnP7P.jpg                                            2
https://pbs.twimg.com/media/CrXhIqBW8AA6Bse.jpg                                            2
https://pbs.twimg.com/media/Ct72q9jWcAAhlnw.jpg                                            2
https://pbs.twimg.com/media/CW88XN4WsAAlo8r.jpg                                            2
https://pbs.twimg.com/media/CeRoBaxWEAABi0X.jpg                                            2
https://pbs.twimg.com/media/Cq9guJ5WgAADfpF.jpg                       

In [165]:
df_image_predictions[df_image_predictions['jpg_url'] == 'https://pbs.twimg.com/media/Co-hmcYXYAASkiG.jpg'].tweet_id

1360    761004547850530816
1522    788070120937619456
Name: tweet_id, dtype: int64

In [166]:
str(df_twitter_archive[df_twitter_archive.tweet_id == 761004547850530816].retweeted_status_id), str(df_twitter_archive[df_twitter_archive.tweet_id == 788070120937619456].retweeted_status_id)


('876   NaN\nName: retweeted_status_id, dtype: float64',
 '686    7.610045e+17\nName: retweeted_status_id, dtype: float64')

Issue summary:
* There are missing record
* reteweeted are present

### Programatic assessment tweet data

In [167]:
df_tweet_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 3 columns):
tweet_id          2337 non-null int64
favorite_count    2337 non-null int64
retweet_count     2337 non-null int64
dtypes: int64(3)
memory usage: 54.9 KB


In [168]:
print('There are {} duplicates in tweet data.'.format(len(df_tweet_data[df_tweet_data.tweet_id.duplicated()])))
print('There are {} missing tweet data.'.format(len(df_twitter_archive) - len(df_tweet_data)))

There are 0 duplicates in tweet data.
There are 19 missing tweet data.


### Assessment summary

#### Quality issues

* Common dataset:
    - there are records related to retweeted tweet
    - missing record in **picture prediction** and **tweet data** datasets
* **twitter achive dataset**:
    - Following columns should be int64 or string instead float64: `in_reply_to_status_id`, `in_reply_to_user_id`
    - Following columns are relative to retweeted and so should be removed: `retweeted_status_id`, `retweeted_status_user_id` and `retweeted_status_timestamp`
    - Column `timestamp` should be of type datetime instead of object
    - Column `name` contain invalid values (ex.: "None" instead of `Nan` when null)
    - Column `source` should be a category.
    - Column `source` should be friendly (remove markup).
    - Column `rating_denominator` contains values other that "10"
    
    
#### Tidiness issues

* The three datasets should be combined into one since they are all part of the same observation unit.
* In **Twitter archive** the columns `doggo`, `floofer`, `pupper` and `puppo` are in fact the same variable and so should be combined into one column


## Clean Data

In [169]:
df_twitter_archive_clean = df_twitter_archive.copy()
df_image_predictions_clean = df_image_predictions.copy()
df_tweet_data_clean = df_tweet_data.copy()

### Clean 2 tidiness issues

#### Issue #1: Tweets is one observation unit

##### Define

Merge all the dataset

##### Code

In [170]:
df_twitter_archive_clean = pd.merge(left=df_twitter_archive_clean,
                                 right=df_tweet_data_clean, left_on='tweet_id', right_on='tweet_id', how='inner')

In [171]:
df_twitter_archive_clean = df_twitter_archive_clean.merge(df_image_predictions_clean, on='tweet_id', how='inner')


##### Test

In [172]:
df_twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2065 entries, 0 to 2064
Data columns (total 30 columns):
tweet_id                      2065 non-null int64
in_reply_to_status_id         23 non-null float64
in_reply_to_user_id           23 non-null float64
timestamp                     2065 non-null object
source                        2065 non-null object
text                          2065 non-null object
retweeted_status_id           75 non-null float64
retweeted_status_user_id      75 non-null float64
retweeted_status_timestamp    75 non-null object
expanded_urls                 2065 non-null object
rating_numerator              2065 non-null int64
rating_denominator            2065 non-null int64
name                          2065 non-null object
doggo                         2065 non-null object
floofer                       2065 non-null object
pupper                        2065 non-null object
puppo                         2065 non-null object
favorite_count                2065 

#### Issue #2: Dog Life Stage

##### Define
Melt column `doggo`, `floofer`, `pupper`, `puppo` into on column name `life_stage`. Take care of the multiple dog stage sometimes possible for one row.

##### Code

In [173]:
to_melt = ['doggo', 'floofer', 'pupper', 'puppo']
to_stay = [x for x in df_twitter_archive_clean.columns.tolist() if x not in to_melt]

df_twitter_archive_clean = pd.melt(df_twitter_archive_clean, id_vars = to_stay, value_vars = to_melt, 
                         var_name = 'stages', value_name = 'life_stage')
df_twitter_archive_clean = df_twitter_archive_clean.drop('stages', 1)
df_twitter_archive_clean = df_twitter_archive_clean.sort_values('life_stage').drop_duplicates('tweet_id', keep = 'last')
df_twitter_archive_clean


,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,life_stage
1988,667435689202614272,NaN,NaN,2015-11-19 20:14:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Ermergerd 12/10 https://t.co/PQni2sjPsm,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667435689...,...,Rottweiler,0.999091,True,miniature_pinscher,0.000450,True,black-and-tan_coonhound,0.000157,True,None
1987,667437278097252352,NaN,NaN,2015-11-19 20:20:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Never seen this breed before. Very pointy pup....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667437278...,...,porcupine,0.989154,False,bath_towel,0.006300,False,badger,0.000966,False,None
1986,667443425659232256,NaN,NaN,2015-11-19 20:44:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Exotic dog here. Long neck. Weird paws. Obsess...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667443425...,...,goose,0.980815,False,drake,0.006918,False,hen,0.005255,False,None
1985,667453023279554560,NaN,NaN,2015-11-19 21:22:56 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Meet Cupcake. I would do unspeakable things fo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667453023...,...,Labrador_retriever,0.825670,True,French_bulldog,0.056639,True,Staffordshire_bullterrier,0.054018,True,None
1984,667455448082227200,NaN,NaN,2015-11-19 21:32:34 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",This is Reese and Twips. Reese protects Twips....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667455448...,...,Tibetan_terrier,0.676376,True,Irish_terrier,0.054933,True,Yorkshire_terrier,0.040576,True,None
1983,667470559035432960,NaN,NaN,2015-11-19 22:32:36 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",This is a northern Wahoo named Kohl. He runs t...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667470559...,...,toy_poodle,0.304175,True,pug,0.223427,True,Lakeland_terrier,0.073316,True,None
1982,667491009379606528,NaN,NaN,2015-11-19 23:53:52 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Two dogs in this one. Both are rare Jujitsu Py...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667491009...,...,borzoi,0.852088,True,ice_bear,0.132264,False,weasel,0.005730,False,None
1981,667495797102141441,NaN,NaN,2015-11-20 00:12:54 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",This is Philippe from Soviet Russia. Commandin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667495797...,...,Chihuahua,0.143957,True,Christmas_stocking,0.118651,False,ski_mask,0.092482,False,None
1980,667502640335572993,NaN,NaN,2015-11-20 00:40:05 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Say hello to Hall and Oates. Oates is winking ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667502640...,...,Labrador_retriever,0.996709,True,golden_retriever,0.001688,True,beagle,0.000712,True,None
1979,667509364010450944,NaN,NaN,2015-11-20 01:06:48 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",This a Norwegian Pewterschmidt named Tickles. ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667509364...,...,beagle,0.636169,True,Labrador_retriever,0.119256,True,golden_retriever,0.082549,True,None


In [174]:
# Visualize the case of multiple dog stage.
df_twitter_archive.groupby(["doggo", "floofer", "pupper", "puppo"]).size().reset_index().rename(columns={0: "count"})


,doggo,floofer,pupper,puppo,count
0,None,None,None,None,1976
1,None,None,None,puppo,29
2,None,None,pupper,None,245
3,None,floofer,None,None,9
4,doggo,None,None,None,83
5,doggo,None,None,puppo,1
6,doggo,None,pupper,None,12
7,doggo,floofer,None,None,1


In [175]:
doggo_puppo = df_twitter_archive.query('doggo == "doggo" and puppo == "puppo"').tweet_id
doggo_pupper = df_twitter_archive.query('doggo == "doggo" and pupper == "pupper"').tweet_id
doggo_floofer = df_twitter_archive.query('doggo == "doggo" and floofer == "floofer"').tweet_id

df_twitter_archive_clean.loc[df_twitter_archive_clean.tweet_id.isin(doggo_puppo), 'life_stage'] = 'doggo,puppo'
df_twitter_archive_clean.loc[df_twitter_archive_clean.tweet_id.isin(doggo_pupper), 'life_stage'] = 'doggo,pupper'
df_twitter_archive_clean.loc[df_twitter_archive_clean.tweet_id.isin(doggo_floofer), 'life_stage'] = 'doggo,floofer'



##### Test

In [176]:
print(df_twitter_archive_clean.life_stage.value_counts())
print(df_twitter_archive_clean.info())

None             1744
pupper            211
doggo              67
puppo              23
doggo,pupper       11
floofer             7
doggo,floofer       1
doggo,puppo         1
Name: life_stage, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2065 entries, 1988 to 6973
Data columns (total 27 columns):
tweet_id                      2065 non-null int64
in_reply_to_status_id         23 non-null float64
in_reply_to_user_id           23 non-null float64
timestamp                     2065 non-null object
source                        2065 non-null object
text                          2065 non-null object
retweeted_status_id           75 non-null float64
retweeted_status_user_id      75 non-null float64
retweeted_status_timestamp    75 non-null object
expanded_urls                 2065 non-null object
rating_numerator              2065 non-null int64
rating_denominator            2065 non-null int64
name                          2065 non-null object
favorite_count               

### Clean 8 quality issues

#### Issue #1: Remove retweeted from twitter archive

##### Define

Remove rows where `retweeted_status_id` is not null.

##### Code

In [177]:
print('number of retweedted {}.'.format(len(df_twitter_archive_clean[df_twitter_archive_clean.retweeted_status_id.notnull()])))
df_twitter_archive_clean = df_twitter_archive_clean[pd.isnull(df_twitter_archive_clean.retweeted_status_id)]
df_twitter_archive_clean

number of retweedted 75.


,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,life_stage
1988,667435689202614272,NaN,NaN,2015-11-19 20:14:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Ermergerd 12/10 https://t.co/PQni2sjPsm,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667435689...,...,Rottweiler,0.999091,True,miniature_pinscher,0.000450,True,black-and-tan_coonhound,0.000157,True,None
1987,667437278097252352,NaN,NaN,2015-11-19 20:20:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Never seen this breed before. Very pointy pup....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667437278...,...,porcupine,0.989154,False,bath_towel,0.006300,False,badger,0.000966,False,None
1986,667443425659232256,NaN,NaN,2015-11-19 20:44:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Exotic dog here. Long neck. Weird paws. Obsess...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667443425...,...,goose,0.980815,False,drake,0.006918,False,hen,0.005255,False,None
1985,667453023279554560,NaN,NaN,2015-11-19 21:22:56 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Meet Cupcake. I would do unspeakable things fo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667453023...,...,Labrador_retriever,0.825670,True,French_bulldog,0.056639,True,Staffordshire_bullterrier,0.054018,True,None
1984,667455448082227200,NaN,NaN,2015-11-19 21:32:34 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",This is Reese and Twips. Reese protects Twips....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667455448...,...,Tibetan_terrier,0.676376,True,Irish_terrier,0.054933,True,Yorkshire_terrier,0.040576,True,None
1983,667470559035432960,NaN,NaN,2015-11-19 22:32:36 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",This is a northern Wahoo named Kohl. He runs t...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667470559...,...,toy_poodle,0.304175,True,pug,0.223427,True,Lakeland_terrier,0.073316,True,None
1982,667491009379606528,NaN,NaN,2015-11-19 23:53:52 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Two dogs in this one. Both are rare Jujitsu Py...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667491009...,...,borzoi,0.852088,True,ice_bear,0.132264,False,weasel,0.005730,False,None
1981,667495797102141441,NaN,NaN,2015-11-20 00:12:54 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",This is Philippe from Soviet Russia. Commandin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667495797...,...,Chihuahua,0.143957,True,Christmas_stocking,0.118651,False,ski_mask,0.092482,False,None
1980,667502640335572993,NaN,NaN,2015-11-20 00:40:05 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Say hello to Hall and Oates. Oates is winking ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667502640...,...,Labrador_retriever,0.996709,True,golden_retriever,0.001688,True,beagle,0.000712,True,None
1979,667509364010450944,NaN,NaN,2015-11-20 01:06:48 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",This a Norwegian Pewterschmidt named Tickles. ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667509364...,...,beagle,0.636169,True,Labrador_retriever,0.119256,True,golden_retriever,0.082549,True,None


##### Test

In [178]:
'number of retweedted {}.'.format(len(df_twitter_archive_clean[df_twitter_archive_clean.retweeted_status_id.notnull()]))

'number of retweedted 0.'

#### Issue #2: Source should be friendly to read

##### Define
Remove the markup from the value.

In [179]:
df_twitter_archive_clean['source'].value_counts(), len(df_twitter_archive_clean)

(<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     1952
 <a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       28
 <a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      10
 Name: source, dtype: int64, 1990)

##### Code

In [180]:
df_twitter_archive_clean.loc[df_twitter_archive_clean.source.str.contains('iphone'),'source'] = 'iPhone'
df_twitter_archive_clean.loc[df_twitter_archive_clean.source.str.contains('vine'),'source'] = 'Vine'
df_twitter_archive_clean.loc[df_twitter_archive_clean.source.str.contains('twitter.com'),'source'] = 'Twitter Web'
df_twitter_archive_clean.loc[df_twitter_archive_clean.source.str.contains('tweetdeck'),'source'] = 'TwitterDeck'

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


##### Test

In [181]:
df_twitter_archive_clean['source'].value_counts(), len(df_twitter_archive_clean)

(iPhone         1952
 Twitter Web      38
 Name: source, dtype: int64, 1990)

#### Issue #3: Fix wrong cateogry type in twitter archive

##### Define

Change type of `source` and `life_stage` to category.

##### Code

In [182]:
df_twitter_archive_clean['source'] = df_twitter_archive_clean['source'].astype('category')
df_twitter_archive_clean['life_stage'] = df_twitter_archive_clean['life_stage'].astype('category')


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


##### Test

In [183]:
df_twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1990 entries, 1988 to 6973
Data columns (total 27 columns):
tweet_id                      1990 non-null int64
in_reply_to_status_id         23 non-null float64
in_reply_to_user_id           23 non-null float64
timestamp                     1990 non-null object
source                        1990 non-null category
text                          1990 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 1990 non-null object
rating_numerator              1990 non-null int64
rating_denominator            1990 non-null int64
name                          1990 non-null object
favorite_count                1990 non-null int64
retweet_count                 1990 non-null int64
jpg_url                       1990 non-null object
img_num                       1990 non-null int64
p1                            1990 n

#### Issue #4: Fix wrong temporal type in twitter archive

##### Define
Change columns `timestamp` and `retweeted_status_timestamp` to Datatime.

##### Code

In [184]:
df_twitter_archive_clean['timestamp'] = pd.to_datetime(df_twitter_archive_clean.timestamp)
df_twitter_archive_clean['retweeted_status_timestamp'] = pd.to_datetime(df_twitter_archive_clean.retweeted_status_timestamp)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


##### Test

In [185]:
df_twitter_archive_clean.dtypes

tweet_id                               int64
in_reply_to_status_id                float64
in_reply_to_user_id                  float64
timestamp                     datetime64[ns]
source                              category
text                                  object
retweeted_status_id                  float64
retweeted_status_user_id             float64
retweeted_status_timestamp    datetime64[ns]
expanded_urls                         object
rating_numerator                       int64
rating_denominator                     int64
name                                  object
favorite_count                         int64
retweet_count                          int64
jpg_url                               object
img_num                                int64
p1                                    object
p1_conf                              float64
p1_dog                                  bool
p2                                    object
p2_conf                              float64
p2_dog    

In [186]:
df_twitter_archive_clean.head(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,life_stage
1988,667435689202614272,NaN,NaN,2015-11-19 20:14:03,iPhone,Ermergerd 12/10 https://t.co/PQni2sjPsm,NaN,NaN,NaT,https://twitter.com/dog_rates/status/667435689...,...,Rottweiler,0.999091,True,miniature_pinscher,0.000450,True,black-and-tan_coonhound,0.000157,True,None
1987,667437278097252352,NaN,NaN,2015-11-19 20:20:22,iPhone,Never seen this breed before. Very pointy pup....,NaN,NaN,NaT,https://twitter.com/dog_rates/status/667437278...,...,porcupine,0.989154,False,bath_towel,0.006300,False,badger,0.000966,False,None
1986,667443425659232256,NaN,NaN,2015-11-19 20:44:47,iPhone,Exotic dog here. Long neck. Weird paws. Obsess...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/667443425...,...,goose,0.980815,False,drake,0.006918,False,hen,0.005255,False,None
1985,667453023279554560,NaN,NaN,2015-11-19 21:22:56,Twitter Web,Meet Cupcake. I would do unspeakable things fo...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/667453023...,...,Labrador_retriever,0.825670,True,French_bulldog,0.056639,True,Staffordshire_bullterrier,0.054018,True,None
1984,667455448082227200,NaN,NaN,2015-11-19 21:32:34,Twitter Web,This is Reese and Twips. Reese protects Twips....,NaN,NaN,NaT,https://twitter.com/dog_rates/status/667455448...,...,Tibetan_terrier,0.676376,True,Irish_terrier,0.054933,True,Yorkshire_terrier,0.040576,True,None


#### Issue #5: Fix rating

##### Define
Assign 10 to the column `rating_denominator`
Extract the decimal of `rating_numerator` from `text` when necessary.

##### Code

In [187]:
# Rating denominator
df_twitter_archive_clean.rating_denominator = 10

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [188]:
# Display which of the rating numerator need to be fix
df_decimals = df_twitter_archive_clean[df_twitter_archive_clean.text.str.contains(r"(\d+\.\d*\/\d+)")]
ratings = df_decimals.text.str.extract('((?:\d+\.)?\d+)\/(\d+)', expand=True).rename(columns={0: 'extract_rating_numerator', 1: 'extract_rating_denominator'})
tweet_id_ratings = pd.concat([df_decimals[['tweet_id','rating_numerator']],ratings], axis=1)
tweet_id_ratings

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


,tweet_id,rating_numerator,extract_rating_numerator,extract_rating_denominator
40,883482846933004288,5,13.5,10
3509,680494726643068929,26,11.26,10
2616,786709082849828864,75,9.75,10
4737,778027034220126208,27,11.27,10


In [189]:
# Apply the fix
for tweet_id in df_decimals.tweet_id:
    df_twitter_archive_clean.loc[df_twitter_archive_clean.tweet_id == tweet_id, 'rating_numerator'] = tweet_id_ratings.extract_rating_numerator


/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


##### Test

In [190]:
df_twitter_archive_clean.rating_denominator.value_counts()

10    1990
Name: rating_denominator, dtype: int64

In [191]:
df_twitter_archive_clean[df_twitter_archive_clean.text.str.contains(r"(\d+\.\d*\/\d+)")][['tweet_id','rating_numerator']]


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


,tweet_id,rating_numerator
40,883482846933004288,13.5
3509,680494726643068929,11.26
2616,786709082849828864,9.75
4737,778027034220126208,11.27


#### Issue #6: Remove columns related to retweet 

##### Define
Remove columns `retweeted_status_id`, `retweeted_status_user_id` and `retweeted_status_timestamp`

##### Code

In [192]:
df_twitter_archive_clean = df_twitter_archive_clean.drop(['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis=1)

##### Test

In [193]:
df_twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1990 entries, 1988 to 6973
Data columns (total 24 columns):
tweet_id                 1990 non-null int64
in_reply_to_status_id    23 non-null float64
in_reply_to_user_id      23 non-null float64
timestamp                1990 non-null datetime64[ns]
source                   1990 non-null category
text                     1990 non-null object
expanded_urls            1990 non-null object
rating_numerator         1990 non-null object
rating_denominator       1990 non-null int64
name                     1990 non-null object
favorite_count           1990 non-null int64
retweet_count            1990 non-null int64
jpg_url                  1990 non-null object
img_num                  1990 non-null int64
p1                       1990 non-null object
p1_conf                  1990 non-null float64
p1_dog                   1990 non-null bool
p2                       1990 non-null object
p2_conf                  1990 non-null float64
p2_dog       

#### Issue #7: Name contains invalid value

##### Define
Replace 'None' by NaN
Replace single letter name by NaN 

##### Code

In [194]:
df_twitter_archive_clean.loc[df_twitter_archive_clean['name'].str.len() < 2, 'name'] = np.NaN
df_twitter_archive_clean.loc[df_twitter_archive_clean['name'] == 'None', 'name'] = np.NaN

##### Test

In [195]:
df_twitter_archive_clean[df_twitter_archive_clean['name'] == "None"]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,...,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,life_stage


In [196]:
df_twitter_archive_clean[df_twitter_archive_clean['name'].str.len() < 2]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,...,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,life_stage


#### Issue #8: Float instead of int

##### Define
Columns `in_reply_to_status_id`, `in_reply_to_user_id` as int.

##### Code

In [197]:
df_twitter_archive_clean['in_reply_to_status_id'].fillna(0, inplace=True)
df_twitter_archive_clean['in_reply_to_status_id'] = df_twitter_archive_clean['in_reply_to_status_id'].astype(int)

df_twitter_archive_clean['in_reply_to_user_id'].fillna(0, inplace=True)
df_twitter_archive_clean['in_reply_to_user_id'] = df_twitter_archive_clean['in_reply_to_user_id'].astype(int)


##### Test

In [198]:
df_twitter_archive_clean.dtypes

tweet_id                          int64
in_reply_to_status_id             int64
in_reply_to_user_id               int64
timestamp                datetime64[ns]
source                         category
text                             object
expanded_urls                    object
rating_numerator                 object
rating_denominator                int64
name                             object
favorite_count                    int64
retweet_count                     int64
jpg_url                          object
img_num                           int64
p1                               object
p1_conf                         float64
p1_dog                             bool
p2                               object
p2_conf                         float64
p2_dog                             bool
p3                               object
p3_conf                         float64
p3_dog                             bool
life_stage                     category
dtype: object

In [199]:
df_twitter_archive_clean.in_reply_to_status_id.value_counts()

0                     1967
667152164079423488       2
667806454573760512       1
692041934689402880       1
670668383499735040       1
675349384339542016       1
674752233200820224       1
684481074559381504       1
674999807681908736       1
729113531270991872       1
703041949650034688       1
671561002136281088       1
759099523532779520       1
733109485275860992       1
703255935813287936       1
691416866452082688       1
675707330206547968       1
746885919387574272       1
674793399141146624       1
675497103322386432       1
684222868335505408       1
855818117272018944       1
671729906628341760       1
Name: in_reply_to_status_id, dtype: int64

### Store the clean data

In [200]:
df_twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1990 entries, 1988 to 6973
Data columns (total 24 columns):
tweet_id                 1990 non-null int64
in_reply_to_status_id    1990 non-null int64
in_reply_to_user_id      1990 non-null int64
timestamp                1990 non-null datetime64[ns]
source                   1990 non-null category
text                     1990 non-null object
expanded_urls            1990 non-null object
rating_numerator         1990 non-null object
rating_denominator       1990 non-null int64
name                     1388 non-null object
favorite_count           1990 non-null int64
retweet_count            1990 non-null int64
jpg_url                  1990 non-null object
img_num                  1990 non-null int64
p1                       1990 non-null object
p1_conf                  1990 non-null float64
p1_dog                   1990 non-null bool
p2                       1990 non-null object
p2_conf                  1990 non-null float64
p2_dog       

In [201]:
df_twitter_archive_clean.to_csv('twitter_archive_master.csv', encoding='utf-8', index=False)